**1. Load Dataset**

In [ ]:
#Pandas: Dataset library
import pandas as pd

#Read dataset
deser_ds = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSylhACMKjuLcz9M7DPkYkAALJMACaSsryKpfbL3IY149_8tfVi0z61Dcg4M267ZaXM-x3MNI-EyoJC/pub?gid=0&single=true&output=csv")

#Print dataset
deser_ds.head()


,Carrera,Sexo,Edad,Semestre,AsignaturasCursadas,AsignaturasAprobadas,CreditosCursados,CreditosAprobados,PromedioUltimoPeriodo,PromedioAcumulado,Duracion,Estado
0,ADMINISTRACION,Femenino,32,10,25,25,25,25,4.25,4.25,5,GRADUADO
1,ADMINISTRACION,Femenino,30,4,29,24,99,82,3.00,3.00,6,ACTIVO
2,ADMINISTRACION,Femenino,35,1,6,6,18,18,3.50,3.50,0,DESERTOR
3,ADMINISTRACION,Femenino,34,4,23,20,63,55,3.00,4.00,4,DESERTOR
4,ADMINISTRACION,Femenino,37,1,6,5,18,15,4.00,4.00,0,DESERTOR


**2. Remove some attribues**

In [ ]:
#Remove some attributes
deser_ds.drop(['Carrera', 'Semestre', 'AsignaturasCursadas', 'AsignaturasAprobadas'], axis=1, inplace=True)

#Print dataset
deser_ds.head()

,Sexo,Edad,CreditosCursados,CreditosAprobados,PromedioUltimoPeriodo,PromedioAcumulado,Duracion,Estado
0,Femenino,32,25,25,4.25,4.25,5,GRADUADO
1,Femenino,30,99,82,3.00,3.00,6,ACTIVO
2,Femenino,35,18,18,3.50,3.50,0,DESERTOR
3,Femenino,34,63,55,3.00,4.00,4,DESERTOR
4,Femenino,37,18,15,4.00,4.00,0,DESERTOR


**3. Standarize dataset**

In [ ]:
from sklearn import preprocessing

#Transforma data to [0-1] scale. 
deser_ds[['Edad', 'CreditosCursados', 'CreditosAprobados', 'PromedioUltimoPeriodo', 'PromedioAcumulado', 'Duracion']] = preprocessing.StandardScaler().fit_transform(deser_ds[['Edad', 'CreditosCursados', 'CreditosAprobados', 'PromedioUltimoPeriodo', 'PromedioAcumulado', 'Duracion']])

#Print dataset
deser_ds.head()

,Sexo,Edad,CreditosCursados,CreditosAprobados,PromedioUltimoPeriodo,PromedioAcumulado,Duracion,Estado
0,Femenino,0.798471,-0.895113,-0.715998,-0.179800,-0.106345,-0.025247,GRADUADO
1,Femenino,0.469929,0.371307,0.353482,-0.189869,-0.121207,0.149821,ACTIVO
2,Femenino,1.291285,-1.014909,-0.847337,-0.185842,-0.115262,-0.900589,DESERTOR
3,Femenino,1.127013,-0.244789,-0.153114,-0.189869,-0.109318,-0.200316,DESERTOR
4,Femenino,1.619827,-1.014909,-0.903626,-0.181814,-0.109318,-0.900589,DESERTOR


**4. Transform attribues**

In [ ]:
from sklearn.preprocessing import LabelEncoder

#Transform YES/NO to 0/1
lb = LabelEncoder() 
deser_ds['Sexo'] = lb.fit_transform(deser_ds['Sexo'])

#Print dataset
deser_ds.head()

,Sexo,Edad,CreditosCursados,CreditosAprobados,PromedioUltimoPeriodo,PromedioAcumulado,Duracion,Estado
0,0,0.798471,-0.895113,-0.715998,-0.179800,-0.106345,-0.025247,GRADUADO
1,0,0.469929,0.371307,0.353482,-0.189869,-0.121207,0.149821,ACTIVO
2,0,1.291285,-1.014909,-0.847337,-0.185842,-0.115262,-0.900589,DESERTOR
3,0,1.127013,-0.244789,-0.153114,-0.189869,-0.109318,-0.200316,DESERTOR
4,0,1.619827,-1.014909,-0.903626,-0.181814,-0.109318,-0.900589,DESERTOR


**5. Train and test split**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize

y = deser_ds['Estado']
X = deser_ds.drop('Estado', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, random_state=0)


**6. Multiple Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0,penalty='l2').fit(X_train, y_train)

#Evaluate Model
from sklearn.metrics import classification_report,confusion_matrix

predictions = clf.predict(X_test)

print("\n=====Confusion Matrix (Logistic Regression):=====\n")
print(confusion_matrix(y_test,predictions))

print("\n=====Classification Report (Logistic Regression):=====\n")
print(classification_report(y_test,predictions))



=====Confusion Matrix (Logistic Regression):=====

[[289  50  45]
 [ 57 241  21]
 [ 21  26 199]]

=====Classification Report (Logistic Regression):=====

              precision    recall  f1-score   support

      ACTIVO       0.79      0.75      0.77       384
    DESERTOR       0.76      0.76      0.76       319
    GRADUADO       0.75      0.81      0.78       246

    accuracy                           0.77       949
   macro avg       0.77      0.77      0.77       949
weighted avg       0.77      0.77      0.77       949



**7. Multiple Decision Tree**

In [ ]:
from sklearn import tree

clf_dt = tree.DecisionTreeClassifier()
clf_dt = clf_dt.fit(X_train, y_train)

#Evaluate Model
from sklearn.metrics import classification_report,confusion_matrix

predictions = clf_dt.predict(X_test)

print("\n=====Confusion Matrix (Decision Tree):=====\n")
print(confusion_matrix(y_test,predictions))

print("\n=====Classification Report (Decision Tree):=====\n")
print(classification_report(y_test,predictions))



=====Confusion Matrix (Decision Tree):=====

[[278  59  47]
 [ 65 236  18]
 [ 43  13 190]]

=====Classification Report (Decision Tree):=====

              precision    recall  f1-score   support

      ACTIVO       0.72      0.72      0.72       384
    DESERTOR       0.77      0.74      0.75       319
    GRADUADO       0.75      0.77      0.76       246

    accuracy                           0.74       949
   macro avg       0.74      0.75      0.74       949
weighted avg       0.74      0.74      0.74       949



**8. Random Forrest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
from scipy import interp

clf_rf = RandomForestClassifier(max_depth=5, random_state=0)
clf_rf = clf_rf.fit(X_train, y_train)

#Evaluate Model
from sklearn.metrics import classification_report,confusion_matrix

predictions = clf_rf.predict(X_test)

print("\n=====Confusion Matrix (Random Forrest):=====\n")
print(confusion_matrix(y_test,predictions))

print("\n=====Classification Report (Random Forrest):=====\n")
print(classification_report(y_test,predictions))



=====Confusion Matrix (Random Forrest):=====

[[272  57  55]
 [ 46 255  18]
 [ 19  15 212]]

=====Classification Report (Random Forrest):=====

              precision    recall  f1-score   support

      ACTIVO       0.81      0.71      0.75       384
    DESERTOR       0.78      0.80      0.79       319
    GRADUADO       0.74      0.86      0.80       246

    accuracy                           0.78       949
   macro avg       0.78      0.79      0.78       949
weighted avg       0.78      0.78      0.78       949

